In [1]:
import re
import nltk
import spacy
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors,FastText,Word2Vec
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
nlp=spacy.load("en_core_web_sm")

In [3]:
text=pd.read_csv('Sentiment Analysis Dataset.csv')
text.columns

Index(['target', 'text'], dtype='object')

In [4]:
x_train=text['text']
y_train=text['target']
len(x_train)

498

In [5]:

x_train_tokenized = []

In [6]:
for sentence in tqdm(x_train):
        doc = nlp(str(sentence)) 
        filtered = [token.lemma_ for token in doc 
                    if not token.is_stop and token.is_alpha
        ]
        x_train_tokenized.append(filtered)

  0%|          | 0/498 [00:00<?, ?it/s]

In [7]:
len(x_train_tokenized)

498

# Word2vec

In [8]:
word2vec = Word2Vec(x_train_tokenized, vector_size=10, window=5, min_count=1,seed=45,workers=4)


In [9]:
len(word2vec.wv)

1555

In [10]:
word2vec.wv.most_similar('love',5)

[('history', 0.8306989073753357),
 ('comment', 0.8241370320320129),
 ('Citi', 0.8110654950141907),
 ('Neutral', 0.7698589563369751),
 ('soul', 0.7655816674232483),
 ('mobile', 0.761715829372406),
 ('Butt', 0.7517223358154297),
 ('person', 0.7494997382164001),
 ('fill', 0.7477650046348572),
 ('safeway', 0.7324135899543762)]

In [11]:
#word2vec.wv['haaaa']

In [12]:

#Odd-One-Out
list=['assesment','Reading','twitter']
word2vec.wv.doesnt_match(list)

'twitter'

In [13]:
#analogy
word2vec.wv.most_similar(positive=['twitter','reading'],negative=['assesment'])

[('word', 0.879749059677124),
 ('moro', 0.8735731244087219),
 ('DOOM', 0.8230501413345337),
 ('wftb', 0.784691333770752),
 ('People', 0.776303231716156),
 ('calculus', 0.7670770883560181),
 ('result', 0.7581081390380859),
 ('bird', 0.75579833984375),
 ('face', 0.7551748156547546),
 ('probably', 0.7521644830703735)]

# Fast Text

In [14]:
fast_text = FastText(
    sentences=x_train_tokenized,
    vector_size=10,
    window=5,
    min_count=1,
    sg=1,
    epochs=20
)


In [15]:
len(fast_text.wv)

1555

# Doc2Vec

In [16]:
tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(x_train)]

In [17]:
d2v_model = Doc2Vec(vector_size=100, window=5, min_count=2, epochs=20)
d2v_model.build_vocab(tagged_data)
d2v_model.train(tagged_data,total_examples=d2v_model.corpus_count,epochs=d2v_model.epochs)

In [21]:
vector = d2v_model.infer_vector(['twiter'])
d2v_model.dv.most_similar([vector])

[('354', 0.11651327461004257),
 ('227', 0.10817532241344452),
 ('170', 0.10799072682857513),
 ('15', 0.10791502147912979),
 ('424', 0.09506969153881073),
 ('293', 0.0950297936797142),
 ('402', 0.09099715948104858),
 ('457', 0.0869593471288681),
 ('235', 0.07728869467973709),
 ('350', 0.06931553035974503)]

# spaCy Pretrained Vectors

In [ ]:
nlp('cat').vector